# RTEM Hackathon notebook

## Import standard libraries

In [1]:
import pandas as pd
import json
import numpy as np

## Import RTEM libraries

In [5]:
# Install onboard client to python, uncomment and run once
'''
!pip install onboard.client
'''

'\n!pip install onboard.client\n'

In [4]:
from onboard.client import RtemClient
from onboard.client.dataframes import points_df_from_streaming_timeseries
from onboard.client.models import PointSelector, TimeseriesQuery, PointData
from datetime import datetime, timezone, timedelta
from typing import List
import pytz

## Load secret

In [5]:
# Read secret from /secrets/ folder
with open('secrets/secrets.txt') as f:
    secret = f.readlines()[0]

# Setup client
client = RtemClient(api_key=secret)

In [11]:
all_buildings = list(pd.DataFrame(client.get_all_buildings())['id'])
print(list(pd.DataFrame(client.get_all_buildings())['id'])) # returns list of buildings that you have access to (you may not have 'Laboratory' in your set)
print(len(all_buildings))

[441, 140, 191, 231, 248, 249, 434, 271, 285, 297, 322, 317, 419, 375, 470, 426, 423, 439, 314, 467, 455, 105, 107, 488, 445, 417, 460, 472, 442, 118, 477, 443, 468, 438, 478, 395, 485, 418, 450, 473, 230, 127, 469, 100, 452, 461, 489, 99, 98, 479, 474, 466, 134, 131, 130, 420, 136, 475, 373, 176, 217, 490, 103, 421, 476, 142, 462, 422, 503, 385, 471, 108, 143, 454, 279, 481, 101, 491, 492, 111, 109, 451, 258, 112, 440, 483, 113, 114, 119, 200, 117, 121, 120, 122, 484, 110, 151, 153, 129, 148, 115, 116, 125, 126, 486, 465, 132, 124, 135, 487, 138, 137, 128, 141, 145, 147, 149, 171, 146, 162, 163, 165, 168, 172, 173, 170, 167, 160, 174, 175, 169, 164, 179, 197, 139, 192, 194, 177, 181, 183, 185, 187, 188, 190, 180, 206, 207, 201, 202, 205, 210, 215, 216, 220, 224, 225, 226, 236, 234, 232, 246, 250, 252, 253, 256, 247, 259, 242, 263, 264, 272, 274, 275, 277, 278, 282, 265, 286, 288, 294, 296, 298, 301, 302, 307, 311, 313, 324, 326, 327, 330, 332, 341, 349, 350, 354, 361, 372, 371, 378, 3

In [15]:
query = PointSelector()
query.point_types     = ['Zone Temperature']        # can list multiple point
query.equipment_types = ['fcu']     # types, equipment types,
query.buildings       = [417, 162, 259]           # buildings, etc.
selection = client.select_points(query)
points = selection["points"]

sensor_metadata = pd.DataFrame(client.get_points_by_ids(points))

#start_date = sensor_metadata.last_updated.apply(lambda x: datetime.fromtimestamp(x/1000, timezone.utc)).min()
#end_date = sensor_metadata.first_updated.apply(lambda x: datetime.fromtimestamp(x/1000, timezone.utc)).max()

tz = pytz.timezone('UTC')
start_date = datetime(2018,1,20,0,0,0).replace(tzinfo=tz)
end_date = datetime(2018,2,20,0,0,0).replace(tzinfo=tz)

print(start_date, end_date)

timeseries_query = TimeseriesQuery(point_ids = selection['points'], start = end_date - timedelta(days=7), end = end_date)
sensor_data = points_df_from_streaming_timeseries(client.stream_point_timeseries(timeseries_query))
print(sensor_data.head())

2018-01-20 00:00:00+00:00 2018-02-20 00:00:00+00:00


ValidationError: 1 validation error for TimeseriesQuery
point_ids
  Exactly one of 'point_ids' or 'selector' is required (type=value_error)

In [14]:
sensor_metadata.head()

,id,building_id,last_updated,first_updated,device,network_device,objectId,name,description,units,tagged_units,raw_unit_id,value,type,point_type_id,measurement_id,datasource_hash,topic,state_text,equip_id
0,199416,162,1.535774e+12,1.517461e+12,,None,,,HV-1 Auditorium_Space Temp,F,None,2,76.9,Zone Temperature,77,1,f2c4e443248ea4b005a09bc1eb5519bb,nyserda+0000082141+ahu1-hv+HV-1 Auditorium_Spa...,None,18310
1,199641,162,1.535127e+12,1.520019e+12,,None,,,Room 186 CUH-22_Space Temp,F,None,2,82.4,Zone Temperature,77,1,e3e5d7e0d12f35007bf67c7ecc209e36,nyserda+0000082141+cuh22-room186+Room 186 CUH-...,None,18329
2,199646,162,1.535127e+12,1.520019e+12,,None,,,Room 184 CUH-23_Space Temp,F,None,2,32,Zone Temperature,77,1,0982edc59ea6a93c2a5bbdc374d682fc,nyserda+0000082141+cuh23-room184+Room 184 CUH-...,None,18330
3,199707,162,1.535774e+12,1.520019e+12,,None,,,Bathroom 116_Space Temp,F,None,2,76.9,Zone Temperature,77,1,ee0c9a22509a975291cbe88431724a52,nyserda+0000082141+fcu-bathroom116+Bathroom 11...,None,18351
4,199712,162,1.535774e+12,1.520019e+12,,None,,,Bathroom 117_Space Temp,F,None,2,76.7,Zone Temperature,77,1,4ec13cb622d4ca8546bb481edadb765a,nyserda+0000082141+fcu-bathroom117+Bathroom 11...,None,18352


In [23]:
pd.json_normalize(client.get_equipment_types()).to_csv("data/equip_types.csv")
pd.json_normalize(client.get_all_point_types()).to_csv("data/point_types.csv")

array(['Occupied Heating Setpoint', 'Outside Air Carbon Dioxide',
       'Return Air Temperature Setpoint',
       'Dual-Temp Coil Discharge Air Temperature',
       'Reheat Coil Discharge Air Temperature',
       'Standby Cooling Setpoint', 'Filter Differential Pressure',
       'Relief Air Damper Position', 'Return Air Humidify Setpoint',
       'Outside Air Temperature', 'Zone Differential Pressure',
       'Return Air Velocity Pressure', 'Zone Carbon Dioxide Setpoint',
       'Outside Air Flow Setpoint', 'Exhaust Air Damper Position',
       'Cooling Coil Discharge Air Temperature',
       'Return Air Static Pressure', 'Unoccupied Heating Setpoint',
       'Return Air Flow Setpoint',
       'Heating Coil Discharge Air Temperature',
       'Mixed Air Damper Position', 'Discharge Air Flow',
       'Bypass Damper Position', 'Preheat Coil Discharge Air Temperature',
       'Outside Air Damper Position', 'Zone Temperature Setpoint',
       'Return Air Dehumidify Setpoint', 'Mixed Air Te